In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 

In [ ]:
# Automated  Basic Cleaning
# All of this should be re-exported back into the CSV before distributing iot 

df = pd.read_csv("Resume.csv", skiprows=[0]).applymap(lambda s:s.lower().strip() if type(s) == str else s)
df = df[df["Start Year"] != 2011]
df["Skills (list all skills as distinct words)"] = df["Skills (list all skills as distinct words)"].str.replace(" ", "")

def isUs(country):
    return type(country) is str and "canada" not in country and (", ca" in country or ", wa" in country or ", ny" in country or ", nj" in country)

# Data Tagging whether internships are from USA or not 
df["is_us_0"] = df["Company Location\n"].map(isUs)
df["is_us_1"] = df["Company Location\n.1"].map(isUs)
df["is_us_2"] = df["Company Location\n.2"].map(isUs)
df["is_us_3"] = df["Company Location\n.3"].map(isUs)
df["is_us_4"] = df["Company Location\n.4"].map(isUs)
df["is_us_5"] = df["Company Location\n.5"].map(isUs)

# TODO: Need to Clean GPA manually from CSV
# TODO: Need to anonymize names
# TODO: Reverse company order

In [ ]:
df.columns

# Step 1 - Data Validation & Cleaning

### Number of Internships vs Start Year 
Student Run - 3 Minutes

In [ ]:
# Construct # of internship Field

df["NumInternships"] = 1
df.loc[~df["Company Name\n.1"].isnull(), "NumInternships"] = 2
df.loc[~df["Company Name\n.2"].isnull(), "NumInternships"] = 3
df.loc[~df["Company Name\n.3"].isnull(), "NumInternships"] = 4
df.loc[~df["Company Name\n.4"].isnull(), "NumInternships"] = 5
df.loc[~df["Company Name\n.5"].isnull(), "NumInternships"] = 6

In [ ]:
# TODO: Figure out outliers with too many internships given start date and manually fix the data. Use df[df["Start Year"] == 2018]
df_clone = df.copy(deep=True)
df_clone = df_clone.dropna(subset=["Start Year"])

X = df_clone["Start Year"]
Y = df_clone["NumInternships"]
plt.scatter( X  + np.random.normal(0, 0.1, len(df_clone)), Y + np.random.normal(0, 0.1, len(df_clone)))
# Linear regression is too hard, don't bother with trying to explain it or demo it 
# plt.plot( X.reshape(-1,1), LinearRegression().fit(X.reshape(-1,1), Y.reshape(-1,1)).predict(X.reshape(-1,1)) )
plt.xlabel("Start Year")
plt.ylabel("# of Internships")

plt.show()

### Sanity Check GPA 
Student Run - 3 Minutes

In [ ]:
# # TODO: Think about whether we want to use .loc instead 

gpa_people = df[df["Average or GPA  "] < 5]
gpa = gpa_people["Average or GPA  "]
gpa.describe()

In [ ]:
grades_people = df[df["Average or GPA  "] > 5]
grades = grades_people["Average or GPA  "]
grades.describe()

# Step 2: Asking dataset what skills I should get 

As a student, I may wonder What skills do I learn if I want to be employed? That's a pretty broad question, so instead we'll answer some more specific q's instead:  

- What skills are most popular among upper year students? 
- What skills are most used in jobs?
- What skills are most likely to get me Cali? <- probably not doing this cause it's lowkey kinda toxic

### What skills do a lot of people know?
Students led - 3 minute 


In [ ]:
skills = df["Skills (list all skills as distinct words)"]
skills = skills.loc[~skills.isnull()]
# This last command is kinda challenging, think about how to show em how to write 
ranked_skills_list = skills.apply(lambda x: pd.value_counts(x.split(","))).sum(axis = 0).sort_values(ascending=False)
ranked_skills_list.head(10)

### What skills do a lot of people use on their jobs?
I really want students to run this analysis on their own, but may be too hard :/ 

In [ ]:
# TODO: Clean up roles - newlines are screwing up our analysis 

roles = df["Job Roles (May be large paragraph)\n"]
roles = roles.append(df["Job Roles (May be large paragraph)\n.1"], ignore_index=True)
roles = roles.append(df["Job Roles (May be large paragraph)\n.2"], ignore_index=True)
roles = roles.append(df["Job Roles (May be large paragraph)\n.3"], ignore_index=True)
roles = roles.append(df["Job Roles (May be large paragraph)\n.4"], ignore_index=True)
roles = roles.append(df["Job Roles (May be large paragraph)\n.5"], ignore_index=True)


roles = roles.loc[~roles.isnull()]
ranked_roles_list = roles.apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).sort_values(ascending=False)
skills_and_duties = pd.concat([ranked_skills_list, ranked_roles_list], axis=1, keys=["skills", "duties"])
skills_and_duties = skills_and_duties.loc[~skills_and_duties["skills"].isnull() & ~skills_and_duties["duties"].isnull()]


In [ ]:
skills_and_duties.sort_values(by=["skills"], ascending=False)

In [ ]:
skills_and_duties.sort_values(by=["duties"], ascending=False)

Key insights : 
- Golang and Android is used by many companies, but many people do not have it as a skill
- For those interested in data, Spark may  be a good first framework to learn 
- C++ is known by many people, but not many jobs are using C++
- Python is quite popular language to learn and also use on the job 
- Docker doesnt seem that popular in terms of co-op jobs (may imply DevOps jobs are rare). Validate this hypothesis? 

# Step 3: Asking dataset where should I apply? 

While developing skills may be helpful, that's not every helpful given that co-op deadline is coming up in XXX weeks. Instead, people may wonder where they should apply given their current situation: 

- Which places hire a lot of interns? (Get hired) 
- Which places hire a lot of first year interns? (some places discriminate against first year kids) 
- Find places where people are likely to return? (Be happy) 

### What places hire a lot of UW kids?

Student Led - 3 Minutes

In [ ]:
companies = df["Company Name\n"]
companies = companies.append(df["Company Name\n.1"])
companies = companies.append(df["Company Name\n.2"])
companies = companies.append(df["Company Name\n.3"])
companies = companies.append(df["Company Name\n.4"])
companies = companies.append(df["Company Name\n.5"])
companies = companies.loc[~companies.isnull()]
companies.value_counts().head(10)

### What places hire a lot of first years?


In [ ]:
one_coops = df[df["NumInternships"] == 1]["Company Name\n"]
two_coops = df[df["NumInternships"] == 2]["Company Name\n.1"]
three_coops = df[df["NumInternships"] == 3]["Company Name\n.2"]
four_coops = df[df["NumInternships"] == 4]["Company Name\n.3"]
five_coops = df[df["NumInternships"] == 5]["Company Name\n.4"]
six_coops = df[df["NumInternships"] == 6]["Company Name\n.5"]

In [ ]:
one_coops.append(two_coops).append(three_coops).append(four_coops).append(five_coops).append(six_coops).value_counts()

### What places hire a lot of interns who return?

In [ ]:
# TODO: Think about whether we want to use .loc instead 
return0 = df[df["Company Name\n.1"] ==  df["Company Name\n"]]["Company Name\n"]
return1 = df[df["Company Name\n.2"] ==  df["Company Name\n.1"]]["Company Name\n.1"]
return2 = df[df["Company Name\n.3"] ==  df["Company Name\n.2"]]["Company Name\n.2"]
return3 = df[df["Company Name\n.4"] ==  df["Company Name\n.3"]]["Company Name\n.3"]
return4 = df[df["Company Name\n.5"] ==  df["Company Name\n.4"]]["Company Name\n.4"]
return0.append(return1).append(return2).append(return3).append(return4).value_counts()

Key insights: 
- Shopify is probably a good place for first years to apply - they hire many waterloo kids, many first year waterloo kids, and people are happy there
- If I don't get google or Cali for my first co-op, that's pretty common. In fact, most first co-ops work in Canada companies 

# Step 4 - Help me understand what a data scientist does 
People like to hype up machine learning. What do machine learning interns do? Is that something I'm interested in? Let's use data to find out. 

### What's a DS anyway? 

In [ ]:
ds_data = df[df["Type\n"]== "ds"][['Skills (list all skills as distinct words)', 'Job Roles (May be large paragraph)\n', 'Job Roles (May be large paragraph)\n.1', 'Job Roles (May be large paragraph)\n.2', 'Job Roles (May be large paragraph)\n.3', 'Job Roles (May be large paragraph)\n.4', 'Job Roles (May be large paragraph)\n.5','Project Description','Project Description.1','Project Description.2','Project Description.3','Project Description.4']]
ds_data.to_csv("ds_data.csv")

### Do I need to move to the States become a DS? 

In [ ]:
ds_people = df[df["Type\n"]=="ds"]
non_ds_people = df[df["Type\n"]!="ds"]


In [ ]:
job0 = ds_people.loc[~ds_people["Company Name\n"].isnull(),"is_us_0"]
job1 = ds_people.loc[~ds_people["Company Name\n.1"].isnull(),"is_us_1"]
job2 = ds_people.loc[~ds_people["Company Name\n.2"].isnull(),"is_us_2"]
job3 = ds_people.loc[~ds_people["Company Name\n.3"].isnull(),"is_us_3"]
job4 = ds_people.loc[~ds_people["Company Name\n.4"].isnull(),"is_us_4"]
job5 = ds_people.loc[~ds_people["Company Name\n.5"].isnull(),"is_us_5"]
ds_jobs = job0.append(job1).append(job2).append(job3).append(job4).append(job5)
ds_jobs.value_counts()

In [ ]:
job0 = non_ds_people.loc[~non_ds_people["Company Name\n"].isnull(),"is_us_0"]
job1 = non_ds_people.loc[~non_ds_people["Company Name\n.1"].isnull(),"is_us_1"]
job2 = non_ds_people.loc[~non_ds_people["Company Name\n.2"].isnull(),"is_us_2"]
job3 = non_ds_people.loc[~non_ds_people["Company Name\n.3"].isnull(),"is_us_3"]
job4 = non_ds_people.loc[~non_ds_people["Company Name\n.4"].isnull(),"is_us_4"]
job5 = non_ds_people.loc[~non_ds_people["Company Name\n.5"].isnull(),"is_us_5"]
non_ds_jobs = job0.append(job1).append(job2).append(job3).append(job4).append(job5)
non_ds_jobs.value_counts()

In [ ]:
4 / (18+4)

In [ ]:
63 / (101 + 63)

In [ ]:
# Compute the number of internships of DS and non DS people 
ds_people["NumInternships"].describe()

### What skills do DS people know? 
There really isn't any new commands here. Just copy paste a previous command 

In [ ]:
skills = ds_people["Skills (list all skills as distinct words)"]
skills = skills.loc[~skills.isnull()]
# This last command is kinda challenging, think about how to show em how to write 
ranked_skills_list = skills.apply(lambda x: pd.value_counts(x.split(","))).sum(axis = 0).sort_values(ascending=False)
ranked_skills_list.head(10)

# Step 4: Understanding limitations in our data 

Briefly talk about how the data was collected (google search "uwaterloo intern filetype:pdf"). Then copy em to a spreadsheet. Probably convert this to an MC question and ask people? 
#### Questions that we cannot answer given our data: 
- Are UW students more likely to work in USA than UofT students? (Missing data) 
- Are DS people more likely to have personal websites than SWE people? (bad data collection method, implies everyone has website)
- Do people with lower grades have less chances of getting PM? (not enough clear grade / GPA data) 
- Do short people have better jobs than tall people? (require causation, correlation is not useful) 
- When applying to Microsoft, does being in SYDE put you at a disadvantage? (causation) 

# Step 5: Summary: 

- Understanding data is not always intuitive. Data Analysts extract value ouf of data, which is not always easy   
- Small improvements may have large downstream impacts (10% drop in rejections -> double # interviews) 
- Pandas and numpy deal with matricies (which are arrays of arrays). Manipulating these arrays help us extract information
- Data analytics typically involves cleaning data before trying to extract value
- Plotting is possible with MatPlot lib 
- We can extract value from both text and non-text data 
- Sometimes we may come to false conclusions from our data. We need to carefully think about our results and try to avoid
- Sometimes, our questions cannot be answered from our data. In which case, we probably will need to collect more data 

In [ ]:
# Alternative way to count skills - This won't be used 
# Clean - Will probably be provided
skillCounter = pd.DataFrame(skills, index=["count"])

for skilllist in df["Skills (list all skills as distinct words)"]:
    for k, v in skillCounter.items():
        if " "+k+" " in skilllist or  " "+k+", " in skilllist:
            skillCounter[k] = skillCounter[k] + 1
    
skillCounter.transpose().sort_values(["count"], ascending=False).head(10)

In [ ]:
# Alternative way to count skills - This won't be used 
# Clean - Will probably be provided
df["Skills (list all skills as distinct words)"].fillna("", inplace=True)

# Get a list of skills
skillsCount = {}
for skilllist in df["Skills (list all skills as distinct words)"]:
    skills = skilllist.split(",")
    if len(skilllist) < 5:
        continue
    for skill in skills:
        skill=skill.strip()
        skillsCount[skill] = skillsCount.get(skill, 0) + 1
skills = {k: 0 for k, v in sorted(skillsCount.items(), key=lambda x:x[1], reverse=True) if v >= 2}


In [ ]:
skillCounter = pd.DataFrame(skills, index=["count"])

jobs = df["Job Roles (May be large paragraph)\n"]
jobs = jobs.append(df["Job Roles (May be large paragraph)\n.1"], ignore_index=True)
jobs = jobs.append(df["Job Roles (May be large paragraph)\n.2"], ignore_index=True)
jobs = jobs.append(df["Job Roles (May be large paragraph)\n.3"], ignore_index=True)
jobs = jobs.append(df["Job Roles (May be large paragraph)\n.4"], ignore_index=True)
jobs = jobs.append(df["Job Roles (May be large paragraph)\n.5"], ignore_index=True)
jobs = jobs.dropna()

for skilllist in jobs:
    skilllist = skilllist.lower()
    for k, v in skillCounter.items():
        if " "+k+", " in skilllist or " "+k+" " in skilllist:
            skillCounter[k] = skillCounter[k] + 1
    
skillCounter.transpose().sort_values(["count"], ascending=False).head(10)